In [72]:
import ollama

import pandas as pd
import time


In [85]:
response = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'What is in this image?',
        'images': ['images/0.jpg']
    }]
)

print(response)

model='llama3.2-vision' created_at='2025-01-06T03:11:41.713341Z' done=True done_reason='stop' total_duration=84690491292 load_duration=5713658042 prompt_eval_count=18 prompt_eval_duration=70255000000 eval_count=106 eval_duration=8498000000 message=Message(role='assistant', content="The painting 'Castle On The Rocks' by Albert Bierstadt depicts a serene landscape featuring a castle situated on rocky outcroppings, set against the backdrop of a vast, misty mountain range. The foreground is filled with lush greenery and possibly some trees or bushes, leading up to the rocky formations where the castle is perched. The painting captures a moment of tranquility, reflecting Bierstadt's fascination with the American West and his ability to convey the majesty and beauty of natural landscapes through his art.", images=None, tool_calls=None)


In [90]:
response['message']['content']

"The painting 'Castle On The Rocks' by Albert Bierstadt depicts a serene landscape featuring a castle situated on rocky outcroppings, set against the backdrop of a vast, misty mountain range. The foreground is filled with lush greenery and possibly some trees or bushes, leading up to the rocky formations where the castle is perched. The painting captures a moment of tranquility, reflecting Bierstadt's fascination with the American West and his ability to convey the majesty and beauty of natural landscapes through his art."

In [9]:
import requests
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_works_by_Albert_Bierstadt"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
table = soup.find_all("table", class_="sortable wikitable")[0]

In [39]:
df = pd.read_html(table.prettify(), extract_links='body')[0]

/var/folders/sv/7jkfjswn39j2qfd9j52qh34c0000gn/T/ipykernel_47378/989770245.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.prettify(), extract_links='body')[0]


In [40]:
df = df.apply(lambda col: [v[0] if v[1] is None else f'https://en.wikipedia.org{v[1]}' for v in col])

In [46]:
df = df[df['Painting'] != '']

In [49]:
df

,Painting,Name,Year,Technique,Dimensions,Current Location
0,https://en.wikipedia.org/wiki/File:Albert_Bier...,Scene in Tyrol,1854,Oil on fiberboard,23.8 cm × 32.4 cm (9.4 in × 12.8 in),https://en.wikipedia.org/wiki/Hirshhorn_Museum...
1,https://en.wikipedia.org/wiki/File:Albert_Bier...,Sunshine and Shadow (or Study for Sunshine a...,1855,Oil on paper mounted on canvas,48.2 cm × 33 cm (19.0 in × 13.0 in),https://en.wikipedia.org/wiki/Newark_Museum
2,https://en.wikipedia.org/wiki/File:The_Old_Mil...,The Old Mill (or Scene in Westphalia ),1855,Oil on canvas,118.8 cm × 94.0 cm (46.8 in × 37.0 in),https://en.wikipedia.org#cite_note-frickoldmill-4
3,https://en.wikipedia.org/wiki/File:Study_of_a_...,Study of a Ewe,1855?,"Drawing: black crayon, red/white chalk on laid...",26.8 cm × 35.7 cm (10.6 in × 14.1 in),https://en.wikipedia.org/wiki/Brooklyn_Museum
4,https://en.wikipedia.org/wiki/File:Albert_Bier...,A Rustic Mill (or A Rustic Mill Farm ),1855,Oil on canvas,375 cm × 279 cm (147.6 in × 109.8 in),https://en.wikipedia.org#cite_note-5
...,...,...,...,...,...,...
192,https://en.wikipedia.orghttp://collection.wmus...,Butterfly,1892,Oil on paper,12.7 cm × 20.3 cm (5.0 in × 8.0 in),https://en.wikipedia.org/wiki/Westmoreland_Mus...
193,https://en.wikipedia.org/wiki/File:Bierstadt_A...,The Landing of Columbus (or Landing of Colum...,1893?,Oil on canvas,182.9 cm × 307.3 cm (72.0 in × 121.0 in),https://en.wikipedia.org/wiki/Newark_Museum
194,https://en.wikipedia.org/wiki/File:The_Landing...,Landing of Columbus at San Salvador (or Land...,1893?,Oil on canvas,203.2 cm × 304.8 cm (80.0 in × 120.0 in),"https://en.wikipedia.org/wiki/Plainfield,_New_..."
195,https://en.wikipedia.org/wiki/File:Bierstadt_A...,The Morteratsch Glacier Upper Engadine Valley ...,1895,Oil on canvas,96.5 cm × 152.4 cm (38.0 in × 60.0 in),https://en.wikipedia.org/wiki/Brooklyn_Museum


In [81]:
urls = list(df['Painting'])

headers = {'User-Agent': 'ArmaanBot/0.1 (armaan.r.butt@gmail.com)'}

In [84]:
for idx, url in enumerate(urls):
    try:
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")
        full_image_body= soup.find(id="file")
        link = full_image_body.find_next('a', href=True)['href']
        link = 'https:' + link
        print(link)
        r = requests.get(link, headers=headers, allow_redirects=True)
        time.sleep(2)
        with open('./images/' + str(idx) + '.jpg', 'wb') as f:
            f.write(r.content)
    except Exception as e:
        print("Error! in " + url)
        print(e)

https://upload.wikimedia.org/wikipedia/commons/9/9b/Albert_Bierstadt_-_Scene_in_the_Tyrol.jpg
https://upload.wikimedia.org/wikipedia/commons/2/22/Albert_Bierstadt_-_Sunlight_and_Shadow_Study.jpg
https://upload.wikimedia.org/wikipedia/commons/a/a2/The_Old_Mill.jpg
https://upload.wikimedia.org/wikipedia/commons/3/35/Study_of_a_Ewe_by_Albert_Bierstadt.jpg
https://upload.wikimedia.org/wikipedia/commons/2/2c/Albert_Bierstadt_A_Rustic_Mill.jpg
https://upload.wikimedia.org/wikipedia/commons/7/74/European_Landscape_by_Albert_Bierstadt.jpg
https://upload.wikimedia.org/wikipedia/commons/5/51/Italian_Costume_Studies_Albert_Bierstadt.jpg
https://upload.wikimedia.org/wikipedia/commons/2/20/Albert_Bierstadt_-_Olevano.jpg
https://upload.wikimedia.org/wikipedia/commons/7/7d/Fishing_Boats_at_Capri.jpg
https://upload.wikimedia.org/wikipedia/commons/e/e8/Martha_Simon%2C_Last_of_the_Narragansetts_oil_1857_Albert_Bierstadt.jpg
https://upload.wikimedia.org/wikipedia/commons/5/57/Albert_Bierstadt_-_Gathering

In [87]:
from glob import glob

image_paths = glob('./images/*.jpg')

In [96]:
IMAGE_PROMPT = """
You are a helpful assist that takes images of paintings and provides an output that will help a RAG-tool find
paintings for users based on mood a feel. Based on the images provide a description that can be used to find
the painting based on mood and other details like place and location. Don't try to guess artist. Only talk about
what you see.

Be concise.
"""

In [97]:
image_descriptions = []

for idx, image_path in enumerate(image_paths):
    response = ollama.chat(
        model='llama3.2-vision',
        messages=[{
            'role': 'system',
            'content': IMAGE_PROMPT
            },
            {
            'role': 'user',
            'content': 'Provide a description of this image.',
            'images': [image_path]
        }]
    )
    description = response['message']['content']
    image_descriptions.append(description)


In [94]:
image_descriptions

['This image is an oil painting by Albert Bierstadt, titled "Sierra Nevada" (1868). The painting depicts a majestic landscape with snow-capped mountains in the background and a serene lake or river in the foreground. A small cabin is nestled among the trees on the right side of the canvas.\n\nThe sky above is painted with soft, white clouds that reflect off the calm waters below. In the distance, one can see the peaks of the Sierra Nevada mountain range rising up towards the horizon. The overall mood of the painting is peaceful and contemplative, inviting the viewer to step into its tranquil atmosphere.',
 'This is an oil painting by Albert Bierstadt titled "Lake Tahoe" (1868). \n\nThe painting depicts Lake Tahoe, located in the Sierra Nevada mountain range on the border between California and Nevada. The lake\'s majestic landscape features a serene and tranquil atmosphere.\n\nIn the foreground, tall pine trees stand along the shoreline, while the calm waters reflect the surrounding mo

In [95]:
df.to_pickle('./painting_data.pkl')

In [101]:
df.to_json(orient='records')

'[{"Painting":"https:\\/\\/en.wikipedia.org\\/wiki\\/File:Albert_Bierstadt_-_Scene_in_the_Tyrol.jpg","Name":"Scene in Tyrol","Year":"1854","Technique":"Oil on fiberboard","Dimensions":"23.8\\u00a0cm \\u00d7\\u00a032.4\\u00a0cm (9.4\\u00a0in \\u00d7\\u00a012.8\\u00a0in)","Current Location":"https:\\/\\/en.wikipedia.org\\/wiki\\/Hirshhorn_Museum_and_Sculpture_Garden"},{"Painting":"https:\\/\\/en.wikipedia.org\\/wiki\\/File:Albert_Bierstadt_-_Sunlight_and_Shadow_Study.jpg","Name":"Sunshine and Shadow  (or  Study for Sunshine and Shadow  ; or  Sunshine and Shadow: Study  )","Year":"1855","Technique":"Oil on paper mounted on canvas","Dimensions":"48.2\\u00a0cm \\u00d7\\u00a033\\u00a0cm (19.0\\u00a0in \\u00d7\\u00a013.0\\u00a0in)","Current Location":"https:\\/\\/en.wikipedia.org\\/wiki\\/Newark_Museum"},{"Painting":"https:\\/\\/en.wikipedia.org\\/wiki\\/File:The_Old_Mill.jpg","Name":"The Old Mill  (or  Scene in Westphalia  )","Year":"1855","Technique":"Oil on canvas","Dimensions":"118.8\\u00

In [102]:
df.head()

,Painting,Name,Year,Technique,Dimensions,Current Location
0,https://en.wikipedia.org/wiki/File:Albert_Bier...,Scene in Tyrol,1854,Oil on fiberboard,23.8 cm × 32.4 cm (9.4 in × 12.8 in),https://en.wikipedia.org/wiki/Hirshhorn_Museum...
1,https://en.wikipedia.org/wiki/File:Albert_Bier...,Sunshine and Shadow (or Study for Sunshine a...,1855,Oil on paper mounted on canvas,48.2 cm × 33 cm (19.0 in × 13.0 in),https://en.wikipedia.org/wiki/Newark_Museum
2,https://en.wikipedia.org/wiki/File:The_Old_Mil...,The Old Mill (or Scene in Westphalia ),1855,Oil on canvas,118.8 cm × 94.0 cm (46.8 in × 37.0 in),https://en.wikipedia.org#cite_note-frickoldmill-4
3,https://en.wikipedia.org/wiki/File:Study_of_a_...,Study of a Ewe,1855?,"Drawing: black crayon, red/white chalk on laid...",26.8 cm × 35.7 cm (10.6 in × 14.1 in),https://en.wikipedia.org/wiki/Brooklyn_Museum
4,https://en.wikipedia.org/wiki/File:Albert_Bier...,A Rustic Mill (or A Rustic Mill Farm ),1855,Oil on canvas,375 cm × 279 cm (147.6 in × 109.8 in),https://en.wikipedia.org#cite_note-5


In [105]:
df_seed = df[['Name', 'Year', 'Technique', 'Dimensions', 'Current Location']]

In [108]:
df_seed.rename(columns={'Name': 'name',
'Year': 'year',
'Technique': 'technique',
'Dimensions': 'dimensions',
'Current Location': 'current_location'}, inplace=True)

/var/folders/sv/7jkfjswn39j2qfd9j52qh34c0000gn/T/ipykernel_47378/3723900884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seed.rename(columns={'Name': 'name',


In [110]:
df_seed.to_json(orient='records')

'[{"name":"Scene in Tyrol","year":"1854","technique":"Oil on fiberboard","dimensions":"23.8\\u00a0cm \\u00d7\\u00a032.4\\u00a0cm (9.4\\u00a0in \\u00d7\\u00a012.8\\u00a0in)","current_location":"https:\\/\\/en.wikipedia.org\\/wiki\\/Hirshhorn_Museum_and_Sculpture_Garden"},{"name":"Sunshine and Shadow  (or  Study for Sunshine and Shadow  ; or  Sunshine and Shadow: Study  )","year":"1855","technique":"Oil on paper mounted on canvas","dimensions":"48.2\\u00a0cm \\u00d7\\u00a033\\u00a0cm (19.0\\u00a0in \\u00d7\\u00a013.0\\u00a0in)","current_location":"https:\\/\\/en.wikipedia.org\\/wiki\\/Newark_Museum"},{"name":"The Old Mill  (or  Scene in Westphalia  )","year":"1855","technique":"Oil on canvas","dimensions":"118.8\\u00a0cm \\u00d7\\u00a094.0\\u00a0cm (46.8\\u00a0in \\u00d7\\u00a037.0\\u00a0in)","current_location":"https:\\/\\/en.wikipedia.org#cite_note-frickoldmill-4"},{"name":"Study of a Ewe","year":"1855?","technique":"Drawing: black crayon, red\\/white chalk on laid paper","dimensions":